# 6. Calibration of double ended measurement with OLS

Use WLS instead. See `examples/notebooks/08Calibrate_double_wls.ipynb`.

A double ended calibration is performed with ordinary least squares. Over all timesteps simultaneous. $\gamma$ and $\int_0^l\alpha$d$x$ remain constant, while $C$ varies over time. The weights are considered equal here and no variance is calculated.

Before starting the calibration procedure, the forward and the backward channel should be aligned.

In [ ]:
import os

from dtscalibration import read_silixa_files
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
filepath = os.path.join('..', '..', 'tests', 'data', 'double_ended2')

ds = read_silixa_files(
    directory=filepath,
    timezone_netcdf='UTC',
    file_ext='*.xml')

ds100 = ds.sel(x=slice(0, 100))  # only calibrate parts of the fiber
sections = {
    'probe1Temperature': [slice(7.5, 17.), slice(70., 80.)],  # cold bath
    'probe2Temperature': [slice(24., 34.), slice(85., 95.)],  # warm bath
    }

In [ ]:
print(ds100.calibration_double_ended.__doc__)

In [ ]:
ds100.calibration_double_ended(sections=sections,
                               method='ols')

After calibration, two data variables are added to the `DataStore` object:
- `tmpf`, temperature calculated along the forward direction
- `tmpb`, temperature calculated along the backward direction

A better estimate, with a lower expected variance, of the temperature along the fiber is the average of the two. We cannot weigh on more than the other, as we do not have more information about the weighing.

In [ ]:
ds1 = ds100.isel(time=0)  # take only the first timestep

ds1.tmpf.plot(linewidth=1, label='User cali. Forward', figsize=(12, 8))  # plot the temperature calibrated by us
ds1.tmpb.plot(linewidth=1, label='User cali. Backward')  # plot the temperature calibrated by us
ds1.tmp.plot(linewidth=1, label='Device calibrated')  # plot the temperature calibrated by the device
plt.legend();

Lets compare our calibrated values with the device calibration. Lets average the temperature of the forward channel and the backward channel first.

In [ ]:
ds1['TMPAVG'] = (ds1.tmpf + ds1.tmpb) / 2
ds1_diff = ds1.tmp - ds1.TMPAVG

ds1_diff.plot(figsize=(12, 8));

The device calibration sections and calibration sections defined by us differ. The device only allows for 2 sections, one per thermometer. And most likely the $\gamma$ is fixed in the device calibration.